# Weight Of Evidence encoding

>The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

It is recommended to use [jupyter](https://jupyter.org/) to run this tutorial.

Binning create buckets of independent variables based on ranking methods. Binning helps us converting continuous variables into categorical ones.

WOE binning Implements a binning of numeric variables and factors with respect to a dichotomous target variable.

```
bin_total = bin_positives + bin_negatives
total_labels = total_positives + total_negatives
bin_WOE = log((bin_positives / total_positives) / (bin_negatives / total_negatives))
bin_iv = ((bin_positives / total_positives) - (bin_negatives / total_negatives)) * bin_woe
```


Currently we provide WOE encoding for vertically partitioned datasets.

Let's first load a sample dataset.

In [1]:
import pandas as pd
import secretflow as sf
from secretflow.data.vertical import VDataFrame
from secretflow.utils.simulation.datasets import load_linear

In [3]:
sf.shutdown()
sf.init(['alice', 'bob'], address='local')
alice, bob = sf.PYU('alice'), sf.PYU('bob')
# similarly for woe in heu
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

/home/isec/anaconda3/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-06 13:40:20,763	INFO worker.py:1724 -- Started a local Ray instance.


In [4]:
parts = {
    bob: (1, 11),
    alice: (11, 22),
}
vdf = load_linear(parts=parts)

INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.


In [8]:
sf.reveal(vdf).values

FedNdarray(partitions={PYURuntime(bob): <secretflow.device.device.pyu.PYUObject object at 0x7fd0e0693bb0>, PYURuntime(alice): <secretflow.device.device.pyu.PYUObject object at 0x7fd0e05e6140>}, partition_way=<PartitionWay.VERTICAL: 'vertical'>)

In [9]:
label_data = vdf['y']
y = sf.reveal(label_data.partitions[alice].data).values

Now, we are ready to perform WOE binning and substitution.

In [10]:
from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning
from secretflow.preprocessing.binning.vert_bin_substitution import VertBinSubstitution

binning = VertWoeBinning(spu)
bin_rules = binning.binning(
    vdf,
    binning_method="chimerge",
    bin_num=4,
    bin_names={alice: ['x14'], bob: ["x5", "x7"]},
    label_name="y",
)

woe_sub = VertBinSubstitution()
vdf = woe_sub.substitution(vdf, bin_rules)

# this is for demo only, be careful with reveal
print(sf.reveal(vdf.partitions[alice].data))
print(sf.reveal(vdf.partitions[bob].data))

INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_binning_pyu.VertWoeBinningPyuWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.preprocessing.binning.vert_woe_binning_pyu.VertWoeBinningPyuWorker'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.data.core.agent.PartitionAgent'> with party alice.


AttributeError: 'tuple' object has no attribute 'partitions'

Sometimes we may need the iv values. Releasing bin ivs will potentially leak label information according to issue https://github.com/secretflow/secretflow/issues/565.
Currently, we choose to save bin iv values in label holders device. It is up to label holder's choice to

1. share no iv information
2. share some chosen iv information

We will demonstrate how to share the feature ivs.

Recall that the woe_rules is a dictionary `{PYU: PYUObject}`, where each `PYUObject` itself is a dictionary of the following type:
```
{
    "variables":[
        {
            "name": str, # feature name
            "type": str, # "string" or "numeric", if feature is discrete or continuous
            "categories": list[str], # categories for discrete feature
            "split_points": list[float], # left-open right-close split points
            "total_counts": list[int], # total samples count in each bins.
            "else_counts": int, # np.nan samples count
            "filling_values": list[float], # woe values for each bins.
            "else_filling_value": float, # woe value for np.nan samples.
        },
        # ... others feature
    ],
    # label holder's PYUObject only
    # warning: giving bin_ivs to other party will leak positive samples in each bin.
    # it is up to label holder's will to give feature iv or bin ivs or all info to workers.
    # for more information, look at: https://github.com/secretflow/secretflow/issues/565

    # in the following comment, by safe we mean label distribution info is not leaked.
    "feature_iv_info" :[
        {
            "name": str, #feature name
            "ivs": list[float], #iv values for each bins, not safe to share with workers in any case.
            "else_iv": float, #iv for nan values, may share to with workers
            "feature_iv": float, #sum of bin_ivs, safe to share with workers when bin num > 2.
        }
    ]
}
```

In [6]:
# alice is label holder
dict_pyu_object = bin_rules[alice]


def extract_name_and_feature_iv(list_of_feature_iv_info):
    return [(d["name"], d["feature_iv"]) for d in list_of_feature_iv_info]


feature_ivs = alice(
    lambda dict_pyu_object: extract_name_and_feature_iv(
        dict_pyu_object["feature_iv_info"]
    )
)(dict_pyu_object)

In [7]:
# we can give the feature_ivs to bob
feature_ivs.to(bob)
# and/or we can reveal it to see it
sf.reveal(feature_ivs)

[('x14', 0.43219177635839423), ('x5', 0.37848298069087766), ('x7', 0)]

In [8]:
feature_iv_info = sf.reveal(feature_ivs)
df = pd.DataFrame.from_records(feature_iv_info, columns=['feature', 'iv'])

How to interpret feature iv?

- Less than 0.02 -> Not useful for prediction
- 0.02 to 0.1 -> Weak predictive Power
- 0.1 to 0.3 -> Medium predictive Power
- 0.3 to 0.5 -> Strong predictive Power
- >0.5	-> Suspicious Predictive Power

let's select top 2 feature iv

In [9]:
print(df.sort_values('iv', ascending=False).head(2))

  feature        iv
0     x14  0.432192
1      x5  0.378483


Congradulations!
In this tutorial we have learnt how to

1. do WOE encoding
2. share some iv information to other parties
3. use feature iv for feature selection
